# 1 Import Libraries



In [ ]:
import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pandas import to_datetime
from sklearn.preprocessing import MinMaxScaler
import seaborn as sns

# 2 Load Data

In [ ]:
# Mengambil file
file_path = 'household_power_consumption.txt'

# Mengambil data dari file
# Menggabungkan kolom Date & Time
df = pd.read_csv(file_path, sep=';', dayfirst=True,
                 infer_datetime_format=True,
                 low_memory=False, na_values=['nan', '?'])

df.head()

,Date,Time,Global_active_power,Global_reactive_power,Voltage,Global_intensity,Sub_metering_1,Sub_metering_2,Sub_metering_3
0,16/12/2006,17:24:00,4.216,0.418,234.84,18.4,0.0,1.0,17.0
1,16/12/2006,17:25:00,5.360,0.436,233.63,23.0,0.0,1.0,16.0
2,16/12/2006,17:26:00,5.374,0.498,233.29,23.0,0.0,2.0,17.0
3,16/12/2006,17:27:00,5.388,0.502,233.74,23.0,0.0,1.0,17.0
4,16/12/2006,17:28:00,3.666,0.528,235.68,15.8,0.0,1.0,17.0


# 3 Preprocess Data

In [ ]:
# Menggabungkan kolom Date & Time
df['Datetime'] = pd.to_datetime(df['Date'] + " " + df['Time'])

df.head()

,Date,Time,Global_active_power,Global_reactive_power,Voltage,Global_intensity,Sub_metering_1,Sub_metering_2,Sub_metering_3,Datetime
0,16/12/2006,17:24:00,4.216,0.418,234.84,18.4,0.0,1.0,17.0,2006-12-16 17:24:00
1,16/12/2006,17:25:00,5.360,0.436,233.63,23.0,0.0,1.0,16.0,2006-12-16 17:25:00
2,16/12/2006,17:26:00,5.374,0.498,233.29,23.0,0.0,2.0,17.0,2006-12-16 17:26:00
3,16/12/2006,17:27:00,5.388,0.502,233.74,23.0,0.0,1.0,17.0,2006-12-16 17:27:00
4,16/12/2006,17:28:00,3.666,0.528,235.68,15.8,0.0,1.0,17.0,2006-12-16 17:28:00


In [ ]:
df.to_csv('house_consumption_melted.csv', index=False)

# 4 Model Build

## Process Melted Dataset

In [ ]:
# Ambil Data
df = pd.read_csv('house_consumption_melted.csv', sep=',', infer_datetime_format=True, header=0)

# Convert 'Datetime' column to datetime type
df['Datetime'] = pd.to_datetime(df['Datetime'])

# Set 'Datetime' as the index
df.set_index('Datetime', inplace=True)

# Group by minute and then resample into 1-hour frequency while summing 'Sub_metering_3'
df_agg = df.resample('1H').agg({'Sub_metering_3': 'sum'})

df_agg

,Sub_metering_3
Datetime,
2006-12-16 17:00:00,607.0
2006-12-16 18:00:00,1012.0
2006-12-16 19:00:00,1001.0
2006-12-16 20:00:00,1007.0
2006-12-16 21:00:00,1033.0
...,...
2008-12-01 19:00:00,1001.0
2008-12-01 20:00:00,1016.0
2008-12-01 21:00:00,1038.0


## Select the Relevant Data

In [ ]:
# Select relevant columns
data = df_agg[['Sub_metering_3']]

# Find the maximum value
max_value = data['Sub_metering_3'].max()

# Find the minimum value
min_value = data['Sub_metering_3'].min()
# Print the result
print(data)
print(f"Maximum value: {max_value}")
print(f"Minimum value: {min_value}")

                     Sub_metering_3
Datetime                           
2006-12-16 17:00:00           607.0
2006-12-16 18:00:00          1012.0
2006-12-16 19:00:00          1001.0
2006-12-16 20:00:00          1007.0
2006-12-16 21:00:00          1033.0
...                             ...
2008-12-01 19:00:00          1001.0
2008-12-01 20:00:00          1016.0
2008-12-01 21:00:00          1038.0
2008-12-01 22:00:00          1036.0
2008-12-01 23:00:00          1053.0

[17191 rows x 1 columns]
Maximum value: 1127.0
Minimum value: 0.0


## Normalize Data

In [ ]:
scaler = MinMaxScaler()
data_normalized = scaler.fit_transform(data)

print(data_normalized)

[[0.53859805]
 [0.89795918]
 [0.88819876]
 ...
 [0.92102928]
 [0.91925466]
 [0.93433895]]


## Splits The Data

In [ ]:
split_time = int(len(data) * 0.8)
train_data = data_normalized[:split_time]
train_target = data_normalized[split_time:]

## Windowed The Dataset

In [ ]:
BATCH_SIZE = 32
N_PAST = 24
N_FUTURE = 24
SHIFT = 1

In [ ]:
def windowed_dataset(series, batch_size, n_past=24, n_future=24, shift=1):
    dataset = tf.data.Dataset.from_tensor_slices(series)
    dataset = dataset.window(size=n_past + n_future, shift=shift, drop_remainder=True)
    dataset = dataset.flat_map(lambda window: window.batch(n_past + n_future))
    dataset = dataset.shuffle(1000)
    dataset = dataset.map(lambda window: (window[:-n_past], window[-n_past:, :1]))
    return dataset.batch(batch_size).prefetch(1)

In [ ]:
# Code to create windowed train and validation datasets.
train_set = windowed_dataset(train_data, BATCH_SIZE, N_PAST, N_FUTURE, SHIFT)
valid_set = windowed_dataset(train_target, BATCH_SIZE, N_PAST, N_FUTURE, SHIFT)

## Build The Model

In [ ]:
model = tf.keras.models.Sequential([
    tf.keras.layers.LSTM(64, return_sequences=True, input_shape=(N_FUTURE,1)),
    tf.keras.layers.LSTM(64),
    tf.keras.layers.Dense(20, activation="relu"),
    tf.keras.layers.Dense(10, activation="relu"),
    tf.keras.layers.Dense(N_FUTURE)
])

# Summary
model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_8 (LSTM)               (None, 24, 64)            16896     
                                                                 
 lstm_9 (LSTM)               (None, 64)                33024     
                                                                 
 dense_12 (Dense)            (None, 20)                1300      
                                                                 
 dense_13 (Dense)            (None, 10)                210       
                                                                 
 dense_14 (Dense)            (None, 24)                264       
                                                                 
Total params: 51694 (201.93 KB)
Trainable params: 51694 (201.93 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


### Compile the Model

In [ ]:
model.compile(optimizer='adam', loss='mae', metrics=["mae"])

### Callback Function

In [ ]:
class myCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
        if logs.get('mae') < 0.22:
            print('MAE terpenuhi')
            self.model.stop_training = True

### Train the Model

In [ ]:
model.fit(train_set, validation_data=valid_set, epochs=300, batch_size=BATCH_SIZE, callbacks=[myCallback()])

Epoch 1/300
268/268 [==============================] - 72s 255ms/step - loss: 0.3008 - mae: 0.3008 - val_loss: 0.0440 - val_mae: 0.0440
Epoch 2/300
268/268 [==============================] - 13s 47ms/step - loss: 0.3007 - mae: 0.3007 - val_loss: 0.0435 - val_mae: 0.0435
Epoch 3/300
268/268 [==============================] - 12s 44ms/step - loss: 0.3008 - mae: 0.3008 - val_loss: 0.0439 - val_mae: 0.0439
Epoch 4/300
268/268 [==============================] - 12s 45ms/step - loss: 0.3007 - mae: 0.3007 - val_loss: 0.0442 - val_mae: 0.0442
Epoch 5/300
268/268 [==============================] - 13s 47ms/step - loss: 0.3007 - mae: 0.3007 - val_loss: 0.0434 - val_mae: 0.0434
Epoch 6/300
268/268 [==============================] - 13s 47ms/step - loss: 0.3007 - mae: 0.3007 - val_loss: 0.0435 - val_mae: 0.0435
Epoch 7/300
268/268 [==============================] - 13s 47ms/step - loss: 0.3007 - mae: 0.3007 - val_loss: 0.0439 - val_mae: 0.0439
Epoch 8/300
268/268 [==============================] -

KeyboardInterrupt: ignored

### Test the Model

In [ ]:
# Make predictions on test data
predictions = model.predict(valid_set)
predictions

106/106 [==============================] - 2s 12ms/step


array([[-1.6084450e-04,  4.6935969e-04,  4.0638438e-04, ...,
         7.1588976e-05,  1.2709454e-05,  2.2224736e-04],
       [-1.6084450e-04,  4.6935969e-04,  4.0638438e-04, ...,
         7.1588976e-05,  1.2709454e-05,  2.2224736e-04],
       [-1.6084450e-04,  4.6935969e-04,  4.0638438e-04, ...,
         7.1588976e-05,  1.2709454e-05,  2.2224736e-04],
       ...,
       [-1.6084450e-04,  4.6935969e-04,  4.0638438e-04, ...,
         7.1588976e-05,  1.2709454e-05,  2.2224736e-04],
       [-1.6084450e-04,  4.6935969e-04,  4.0638438e-04, ...,
         7.1588976e-05,  1.2709454e-05,  2.2224736e-04],
       [-1.6084450e-04,  4.6935969e-04,  4.0638438e-04, ...,
         7.1588976e-05,  1.2709454e-05,  2.2224736e-04]], dtype=float32)

In [ ]:
# Inverse transform predictions to original scale
predictions = scaler.inverse_transform(predictions)
data_normalized_inverse = scaler.inverse_transform(data_normalized[:split_time])

In [ ]:
print(predictions)

[[-0.48671028 -0.27496964  0.1533839  ... -0.2574001   0.4031872
   0.31027734]
 [-0.43540525 -0.5482515  -0.07525321 ... -0.39453986  0.38964623
   0.22514063]
 [-0.5144385  -0.10226783  0.2933691  ... -0.16975215  0.425948
   0.38583735]
 ...
 [-0.4298508  -0.40114212  0.01600378 ... -0.31377834  0.4973059
   0.42472473]
 [-0.4298508  -0.40114212  0.01600378 ... -0.31377834  0.4973059
   0.42472473]
 [-0.4298508  -0.40114212  0.01600378 ... -0.31377834  0.4973059
   0.42472473]]


In [ ]:
print(data_normalized_inverse)

[[ 607.]
 [1012.]
 [1001.]
 ...
 [ 659.]
 [   0.]
 [ 356.]]


# 5 Generate Save Model

## Save the Model

In [ ]:
run_model = tf.function(lambda x: model(x))
STEPS = 1
INPUT_SIZE = 24
concrete_func = run_model.get_concrete_function(
    tf.TensorSpec([STEPS, INPUT_SIZE], model.inputs[0].dtype))

# model directory.
MODEL_DIR = "model_kWh_5"
model.save(MODEL_DIR, save_format="tf", signatures=concrete_func)

converter = tf.lite.TFLiteConverter.from_saved_model(MODEL_DIR)
tflite_model = converter.convert()

## Convert the Model to TFLITE

In [ ]:
import pathlib
tflite_model_file = pathlib.Path('model_kWh_5.tflite')
tflite_model_file.write_bytes(tflite_model)

213228